In [30]:
"""
First lets get the common libraries here in play
"""
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [31]:
data = pd.read_csv('./data/MTA_Subway_Stations_20241127.csv')

data.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)


In [32]:
data = data.drop(columns=['CBD', 
                          'Structure', 
                          'ADA', 
                          'ADA Northbound', 
                          'ADA Southbound', 
                          'ADA Notes'])

In [33]:
data.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,40.775036,-73.912034,Last Stop,Manhattan,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,N W,40.770258,-73.917843,Astoria,Manhattan,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,N W,40.766779,-73.921479,Astoria,Manhattan,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,N W,40.761820,-73.925508,Astoria,Manhattan,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,N W,40.756804,-73.929575,Astoria,Manhattan,POINT (-73.929575 40.756804)


In [40]:
data = data[data['Borough'] != 'SI']
last_stops = data[data['North Direction Label'] == 'Last Stop']

last_stops.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,40.775036,-73.912034,Last Stop,Manhattan,POINT (-73.912034 40.775036)
114,L01,115,618,BMT,Canarsie,8 Av,M,L,40.739777,-74.002578,Last Stop,Brooklyn,POINT (-74.002578 40.739777)
138,S01,139,627,BMT,Franklin Shuttle,Franklin Av,Bk,S,40.680596,-73.955827,Last Stop,Prospect Park,POINT (-73.955827 40.680596)
141,A02,143,143,IND,8th Av - Fulton St,Inwood-207 St,M,A,40.868072,-73.919899,Last Stop,Downtown,POINT (-73.919899 40.868072)
210,D01,210,210,IND,Concourse,Norwood-205 St,Bx,D,40.874811,-73.878855,Last Stop,Manhattan,POINT (-73.878855 40.874811)


In [49]:
f_train = data[data['Daytime Routes'].str.contains('F')]

f_train

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,Georeference
56,D42,57,57,BMT,Broadway - Brighton,W 8 St-NY Aquarium,Bk,F Q,40.576127,-73.975939,Manhattan,Coney Island,POINT (-73.975939 40.576127)
57,D43,58,58,BMT,Sea Beach / West End / Culver / Brighton,Coney Island-Stillwell Av,Bk,D F N Q,40.577422,-73.981233,Manhattan,Last Stop,POINT (-73.981233 40.577422)
167,D20,167,167,IND,6th Av - Culver,W 4 St-Wash Sq,M,B D F M,40.732338,-74.000495,Uptown,Downtown,POINT (-74.000495 40.732338)
174,A41,174,636,IND,8th Av - Fulton St,Jay St-MetroTech,Bk,A C F,40.692338,-73.987342,Manhattan,Outbound,POINT (-73.987342 40.692338)
221,B04,221,221,IND,63rd St,21 St-Queensbridge,Q,F,40.754203,-73.942836,Outbound,Manhattan,POINT (-73.942836 40.754203)
222,B06,222,222,IND,63rd St,Roosevelt Island,M,F,40.759145,-73.953260,Queens,Downtown,POINT (-73.95326 40.759145)
223,B08,223,223,IND,63rd St,Lexington Av/63 St,M,F Q,40.764629,-73.966113,Uptown & Queens,Downtown,POINT (-73.966113 40.764629)
224,B10,224,224,IND,6th Av - Culver,57 St,M,F,40.763972,-73.977450,Uptown,Downtown,POINT (-73.97745 40.763972)
225,D15,225,225,IND,6th Av - Culver,47-50 Sts-Rockefeller Ctr,M,B D F M,40.758663,-73.981329,Uptown,Downtown,POINT (-73.981329 40.758663)
226,D16,226,609,IND,6th Av - Culver,42 St-Bryant Pk,M,B D F M,40.754222,-73.984569,Uptown,Downtown,POINT (-73.984569 40.754222)


Lets find a way to re order them to how the actual train line is.

In [52]:
def nearest_neighbor_path():
    return ""

def swap_rows(df, idx1, idx2):
    """Swap two rows in a DataFrame"""
    df.iloc[idx1], df.iloc[idx2] = df.iloc[idx2].copy(), df.iloc[idx1].copy()
    return df

swap_rows(f_train, 0, 1)

coords = f_train[['GTFS Latitude', 'GTFS Longitude']].values
distances = pdist(coords)
distance_matrix = squareform(distances)

#print(coords)
#print(distances)
print(distance_matrix)

[[0.         0.00545009 0.15610891 ... 0.20140791 0.19875542 0.19161732]
 [0.00545009 0.         0.15812929 ... 0.19858524 0.19611111 0.19035067]
 [0.15610891 0.15812929 0.         ... 0.16377109 0.15633697 0.11009047]
 ...
 [0.20140791 0.19858524 0.16377109 ... 0.         0.0079427  0.06098474]
 [0.19875542 0.19611111 0.15633697 ... 0.0079427  0.         0.05305171]
 [0.19161732 0.19035067 0.11009047 ... 0.06098474 0.05305171 0.        ]]
